<a href="https://colab.research.google.com/github/oussamazrx/oussamazrx/blob/main/text_classification_arabert_without_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch, os
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
import shutil
import pandas as pd
import subprocess

# Extract 7z file using 7z command-line tool
def extract_7z(file_path):
    subprocess.run(['7z', 'x', file_path])

# Function to read text files from directories
def read_text_files(directory):
    text_data = []
    for subdir, dirs, files in os.walk(directory):
        if files:
            label = os.path.basename(subdir)
            for file in files:
                if file.endswith(".txt"):
                    file_path = os.path.join(subdir, file)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        text = f.read()
                        text_data.append((text, label))
    return text_data

# Extract the 7z file
extract_7z("/content/cnn-arabic-utf8.7z")

# Read text files and create dataset
data = read_text_files("./cnn-arabic-utf8")

# Create DataFrame
df = pd.DataFrame(data, columns=['text', 'label'])

# Save DataFrame to CSV file
df.to_csv("dataset.csv", index=False)


In [ ]:
labels = df['label'].unique().tolist()
labels = [s.strip() for s in labels ]
labels

['sport', 'middle_east', 'scitech', 'business', 'world', 'entertainment']

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
import torch.nn.functional as F
import torch
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"{torch.cuda.get_device_name(0)} GPU is used")
    !nvidia-smi
else:
    print("No GPU is detected")
    device = torch.device("cpu")

Tesla T4 GPU is used
Thu May  2 17:56:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----

In [ ]:
model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=11)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

target_encoder = LabelEncoder().fit(df.label)
target = target_encoder.transform(df.label)

X_train, X_test, y_train, y_test = train_test_split(df.drop(["label"], axis=1), target, test_size=0.2, random_state=10, stratify=target, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=10, stratify=y_train, shuffle=True)

In [ ]:
MAX_LEN = 256

train_encoding = tokenizer(X_train["text"].to_list(), padding=True, truncation=True, max_length=MAX_LEN)
val_encoding = tokenizer(X_val["text"].to_list(), padding=True, truncation=True, max_length=MAX_LEN)
test_encoding = tokenizer(X_test["text"].to_list(), padding=True, truncation=True, max_length=MAX_LEN)

tr_input_ids, tr_attn_mask = torch.tensor(train_encoding.input_ids), torch.tensor(train_encoding.attention_mask)
val_input_ids, val_attn_mask = torch.tensor(val_encoding.input_ids), torch.tensor(val_encoding.attention_mask)
ts_input_ids, ts_attn_mask = torch.tensor(test_encoding.input_ids), torch.tensor(test_encoding.attention_mask)

In [ ]:
BATCH_SIZE = 32

train_data = TensorDataset(tr_input_ids, tr_attn_mask, torch.tensor(y_train))
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE)

val_data = TensorDataset(val_input_ids, val_attn_mask, torch.tensor(y_val))
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE)

test_data = TensorDataset(ts_input_ids, ts_attn_mask, torch.tensor(y_test))
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
optimizer = optim.AdamW(bert_model.parameters(), lr=1e-5, weight_decay=0.02)
lr_scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.4)

In [ ]:
import numpy as np

def flat_accuracy(y_pred, y):
    flat_y_pred = np.argmax(y_pred, axis=1)
    return np.sum(flat_y_pred == y) / len(y)

EPOCHS = 2
bert_model.to(device)
for e in range(EPOCHS):
    print("="*40)
    print(f'Epoch: {e+1}/{EPOCHS} | lr: {optimizer.param_groups[0]["lr"]}')
    print("="*40)
    bert_model.train()

    train_accuracy = []
    # going through batches
    for s, batch in enumerate(train_dataloader):
        input_ids, attn_mask, y = batch
        optimizer.zero_grad()

        # forward
        output = bert_model(input_ids.to(device), attention_mask=attn_mask.to(device), labels=y.to(device))

        # backward
        output.loss.backward()

        # update params
        optimizer.step()

        # tracking vars
        accuracy = flat_accuracy(output.logits.detach().cpu().numpy(), y.to("cpu").numpy())
        train_accuracy.append(accuracy)

        print(f'Step: {s:^5} | Loss: {output.loss.item()}')

    print("-"*40)
    print(f'Train Accuracy: {sum(train_accuracy) / len(train_accuracy) * 100:.2f} %')
    lr_scheduler.step()

    bert_model.eval()
    val_accuracy = []
    for batch in val_dataloader:
        input_ids, attn_mask, y = batch
        with torch.no_grad():
            output = bert_model(input_ids.to(device), attention_mask=attn_mask.to(device))

        accuracy = flat_accuracy(output.logits.detach().cpu().numpy(), y.to("cpu").numpy())
        val_accuracy.append(accuracy)
    print(f"Val Accuracy: {sum(val_accuracy) / len(val_accuracy) * 100:.2f} %")
    print("-"*40)

bert_model.save_pretrained("path/to/sequence-classification-model")

Epoch: 1/2 | lr: 1e-05
Step:   0   | Loss: 2.347080707550049
Step:   1   | Loss: 2.2863118648529053
Step:   2   | Loss: 2.272334098815918
Step:   3   | Loss: 2.1567559242248535
Step:   4   | Loss: 2.1727218627929688
Step:   5   | Loss: 2.065013885498047
Step:   6   | Loss: 2.0249414443969727
Step:   7   | Loss: 1.932782769203186
Step:   8   | Loss: 2.0317647457122803
Step:   9   | Loss: 1.9497205018997192
Step:  10   | Loss: 1.7908647060394287
Step:  11   | Loss: 1.759268879890442
Step:  12   | Loss: 1.757039189338684
Step:  13   | Loss: 1.7640714645385742
Step:  14   | Loss: 1.7654403448104858
Step:  15   | Loss: 1.6687078475952148
Step:  16   | Loss: 1.6125415563583374
Step:  17   | Loss: 1.745836615562439
Step:  18   | Loss: 1.600143313407898
Step:  19   | Loss: 1.600611686706543
Step:  20   | Loss: 1.6984905004501343
Step:  21   | Loss: 1.5359529256820679
Step:  22   | Loss: 1.6130602359771729
Step:  23   | Loss: 1.571523666381836
Step:  24   | Loss: 1.3426811695098877
Step:  25   

In [ ]:
def bert_model_predict(model, dataloader, score="flat"):
    bert_model.eval()
    pred_probs = []
    logits_list = []

    for batch in dataloader:
        input_ids, attn_mask, _ = batch

        with torch.no_grad():
            logits = model(input_ids.to(device), attn_mask.to(device)).logits

        probs = F.softmax(logits, dim=1)
        pred_probs.append(probs)
        logits_list.append(logits)

    pred_probs = torch.cat(pred_probs).cpu().numpy()

    if score=="probs":
        return pred_probs
    elif score=="logits":
        return torch.cat(logits_list).cpu().numpy()
    else:
        pred_flat = np.argmax(pred_probs, axis=1)
        return pred_flat

In [ ]:
bert_pred_flat = bert_model_predict(bert_model, test_dataloader, score="flat")
print(classification_report(y_test ,bert_pred_flat, labels=target, target_names=target_encoder.inverse_transform([0,1,2,3,4,5])))

               precision    recall  f1-score   support

     business       0.98      1.00      0.99       152
entertainment       0.98      1.00      0.99       152
  middle_east       0.98      1.00      0.99       152
      scitech       0.98      1.00      0.99       152
        sport       0.98      1.00      0.99       152
        world       0.98      1.00      0.99       152

     accuracy                           0.94    988082
    macro avg       0.93      0.93      0.93    988082
 weighted avg       0.94      0.93      0.94    988082



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2326: UserWarning: labels size, 5070, does not match size of target_names, 6
  warnings.warn(


In [ ]:
def evaluate_single_instance(model, tokenizer, text):
    encoding = tokenizer(text, padding=True, truncation=True, max_length=MAX_LEN, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_mask).logits
    probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
    return probs

# Interactive evaluation loop
while True:
    text = input("Enter text to evaluate (type 'quit' to exit): ")
    if text.lower() == 'quit':
        break
    probabilities = evaluate_single_instance(bert_model, tokenizer, text)
    predicted_class = np.argmax(probabilities)
    predicted_label = target_encoder.inverse_transform([predicted_class])[0]
    print(f"Predicted class: {predicted_label}")
    for label, prob in zip(target_encoder.classes_, probabilities):
        print(f"{label}: {prob:.2f}")
